In [ ]:
#NER : Named entity recognition

In [ ]:
bill gates founded microsoft

In [ ]:
# spacy

In [2]:
import spacy

In [3]:
ner = spacy.load('en_core_web_sm')

In [4]:
text = "bill gates founded microsoft"

doc = ner(text)

print(doc.ents)

(bill gates, microsoft)


In [8]:
for entity in doc.ents:
  print(entity.text, '=>',entity.label_,  '=>',spacy.explain(entity.label_))

bill gates => PERSON => People, including fictional
microsoft => ORG => Companies, agencies, institutions, etc.


In [10]:
for ent in ner.pipe_labels['ner']:
  print(ent, '=>', spacy.explain(ent))

CARDINAL => Numerals that do not fall under another type
DATE => Absolute or relative dates or periods
EVENT => Named hurricanes, battles, wars, sports events, etc.
FAC => Buildings, airports, highways, bridges, etc.
GPE => Countries, cities, states
LANGUAGE => Any named language
LAW => Named documents made into laws.
LOC => Non-GPE locations, mountain ranges, bodies of water
MONEY => Monetary values, including unit
NORP => Nationalities or religious or political groups
ORDINAL => "first", "second", etc.
ORG => Companies, agencies, institutions, etc.
PERCENT => Percentage, including "%"
PERSON => People, including fictional
PRODUCT => Objects, vehicles, foods, etc. (not services)
QUANTITY => Measurements, as of weight or distance
TIME => Times smaller than a day
WORK_OF_ART => Titles of books, songs, etc.


In [4]:
text = "we are learning nlp in IBM"

doc = ner(text)

print(doc.ents)

(IBM,)


In [11]:
from spacy.pipeline import EntityRuler

In [5]:
nlp = spacy.load("en_core_web_sm")

# pattern define
patterns = [{"label": "ORG", "pattern" : [{"TEXT" : "IBM"}]}]

#add entityruler component to pipeline
ruler = ner.add_pipe("entity_ruler", before="ner")

#add pattern in ruler
ruler.add_patterns(patterns)

text = "we are learning nlp in IBM"

doc = ner(text)

for entity in doc.ents:
  print(entity.text, '=>',entity.label_,  '=>',spacy.explain(entity.label_))

IBM => ORG => Companies, agencies, institutions, etc.


In [20]:
ner = spacy.load("en_core_web_sm")

# pattern define
patterns = [{"label": "PERSON", "pattern" : [{"TEXT" : "narayan"}, {"TEXT" : "murty"}]},
            {"label": "ORG", "pattern" : [{"TEXT" : "infosys"}]},
            {"label": "PERSON", "pattern" : [{"TEXT" : "Murty"}]}]

#add entityruler component to pipeline
ruler = ner.add_pipe("entity_ruler", before="ner")

#add pattern in ruler
ruler.add_patterns(patterns)

text = "Murty founded infosys"

doc = ner(text)

for entity in doc.ents:
  print(entity.text, '=>',entity.label_,  '=>',spacy.explain(entity.label_))

Murty => PERSON => People, including fictional
infosys => ORG => Companies, agencies, institutions, etc.


In [24]:
ner = spacy.load("en_core_web_sm")

# pattern define
patterns = [{"label": "PHONE", "pattern" : [{"TEXT" : {"REGEX": "[0-9]{10}"}}]},
            {"label": "EMAIL", "pattern" : [{"TEXT" : {"REGEX": "[a-z]@[a-z.]"}}]}]

#add entityruler component to pipeline
ruler = ner.add_pipe("entity_ruler", before="ner")

#add pattern in ruler
ruler.add_patterns(patterns)

text = "narayan murty founded infosys and his contact is 9876543210 and email is murty@infosys.com"

doc = ner(text)

for entity in doc.ents:
  print(entity.text, '=>',entity.label_,  '=>')

narayan murty => PERSON =>
9876543210 => PHONE =>
murty@infosys.com => EMAIL =>


##Finetuning

In [ ]:
#labelling tools

# https://tecoholic.github.io/ner-annotator/ --> opensource

In [25]:
import spacy
from spacy.tokens import DocBin

In [26]:
nlp = spacy.blank('en')
db = DocBin()

In [27]:
import json

f = open("/content/annotations.json")

TRAIN_DATA = json.load(f)

In [28]:
TRAIN_DATA

{'classes': ['VIRUS', 'DIESESE'],
 'annotations': [["Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the virus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019.[6] The disease quickly spread worldwide, resulting in the COVID-19 pandemic.\r\n\r\nThe symptoms of COVID‑19 are variable but often include fever,[7] cough, headache,[8] fatigue, breathing difficulties, loss of smell, and loss of taste.[9][10][11] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[12][13] Of those who develop symptoms noticeable enough to be classified as patients, most (81%) develop mild to moderate symptoms (up to mild pneumonia), while 14% develop severe symptoms (dyspnea, hypoxia, or more than 50% lung involvement on imaging), and 5% develop critical symptoms (respiratory failure, shock, or multiorgan dysfunction).[14] Older people are at a higher risk of d

In [39]:
for text, annot in TRAIN_DATA["annotations"]:
  doc = nlp.make_doc(text)
  ent = []
for start, end, label in annot['entities']:
span = doc.char_span(start, end, label, alignment_mode='contract')
if span is None:
  print('none')
else:
  ent.append(span)
doc.ents = ent
db.add(doc)
db.to_disk("training_data.spacy")

none
none


In [35]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2024-01-30 02:43:38.645449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 02:43:38.645522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 02:43:38.647044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 02:43:40.470098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- 

In [40]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2024-01-30 02:47:36.402008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 02:47:36.402072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 02:47:36.403435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 02:47:37.787665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['to

In [33]:
!python -m spacy train --help

2024-01-30 02:41:53.876023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 02:41:53.876120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 02:41:53.880005: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 02:41:55.670073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
                                                                                                    
 Usage: python -m spacy train [OPTIONS] CONFIG_PATH                                                 
                                       

In [ ]:
##making inferences on trained model

In [41]:
trained_ner = spacy.load("/content/model-best")

In [46]:
text = "COVID-19 is a contagious dieses."

doc = trained_ner(text)

In [47]:
doc.ents

()

##Transformers

In [48]:
!pip install transformers

In [50]:
from transformers import pipeline

In [51]:
ner_transformer = pipeline(model="dslim/bert-base-NER-uncased")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#IOB format :
I --> inside
o --> outside
B --> beginning

In [52]:
text = "bill gates founded microsoft"
ner_transformer(text)

[{'entity': 'B-PER',
  'score': 0.99582714,
  'index': 1,
  'word': 'bill',
  'start': 0,
  'end': 4},
 {'entity': 'I-PER',
  'score': 0.99281466,
  'index': 2,
  'word': 'gates',
  'start': 5,
  'end': 10},
 {'entity': 'B-ORG',
  'score': 0.98988754,
  'index': 4,
  'word': 'microsoft',
  'start': 19,
  'end': 28}]

In [53]:
text = "narayan murthy founded infosys"
ner_transformer(text)

[{'entity': 'B-PER',
  'score': 0.99548846,
  'index': 1,
  'word': 'narayan',
  'start': 0,
  'end': 7},
 {'entity': 'I-PER',
  'score': 0.9973943,
  'index': 2,
  'word': 'mu',
  'start': 8,
  'end': 10},
 {'entity': 'I-PER',
  'score': 0.984729,
  'index': 3,
  'word': '##rth',
  'start': 10,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.98603475,
  'index': 4,
  'word': '##y',
  'start': 13,
  'end': 14},
 {'entity': 'B-ORG',
  'score': 0.99705446,
  'index': 6,
  'word': 'info',
  'start': 23,
  'end': 27},
 {'entity': 'I-ORG',
  'score': 0.9965019,
  'index': 7,
  'word': '##sy',
  'start': 27,
  'end': 29},
 {'entity': 'I-ORG',
  'score': 0.9972198,
  'index': 8,
  'word': '##s',
  'start': 29,
  'end': 30}]